In [2]:
# import libraries
from numpy import *
import pandas as pd
from yahoo_fin.stock_info import get_data

import json
import datetime
import matplotlib.pyplot as plt

import seaborn as sns

plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('fivethirtyeight')
########### FOCUS ON BIOTECH INDUSTRY #############
###### WE ONLY SELECT THE FIRMS THAT HAVE LESS THAN $1 BILLION MARKET CAP TO CAPTURE MORE CHARACTERSITICS 

# import data from yahoo finance
bio_stocks = ['BGNE', 'RPRX', 'ARGX', 'MRNA', 'HLN', 'ZTS', 'VRTX', 'GILD']

# create the adjclose dataframe
df_bio_adjclose = pd.DataFrame()
# create the volume dataframe
df_bio_vol = pd.DataFrame()

for stock in bio_stocks:
    adjclose = get_data(stock, start_date="10/31/2018", end_date="10/31/2023", index_as_date=True, interval="1d")[
        'adjclose']
    vol = get_data(stock, start_date="10/31/2018", end_date="10/31/2023", index_as_date=True, interval="1d")['volume']

    df_bio_adjclose = pd.concat([df_bio_adjclose, adjclose], axis=1)
    df_bio_vol = pd.concat([df_bio_vol, vol], axis=1)

df_bio_adjclose.columns = bio_stocks
df_bio_vol.columns = bio_stocks

# impute the na wiht the average of the nearest 50 values or should we use simulation to impute?
rprx_ave = mean(df_bio_adjclose['RPRX'][407:457])
mrna_ave = mean(df_bio_adjclose['MRNA'][25:75])
hln_ave = mean(df_bio_adjclose['HLN'][937:987])

df_bio_adjclose['RPRX'] = df_bio_adjclose['RPRX'].fillna(rprx_ave)
df_bio_adjclose['MRNA'] = df_bio_adjclose['MRNA'].fillna(mrna_ave)
df_bio_adjclose['HLN'] = df_bio_adjclose['HLN'].fillna(hln_ave)

rprx_vol = mean(df_bio_vol['RPRX'][407:457])
mrna_vol = mean(df_bio_vol['MRNA'][25:75])
hln_vol = mean(df_bio_vol['HLN'][937:987])

df_bio_vol['RPRX'] = df_bio_vol['RPRX'].fillna(rprx_vol)
df_bio_vol['MRNA'] = df_bio_vol['MRNA'].fillna(mrna_vol)
df_bio_vol['HLN'] = df_bio_vol['HLN'].fillna(hln_vol)
# compute the statistics for adjclose
mean = df_bio_adjclose.mean()
sd = sqrt(df_bio_adjclose.var())
skew = df_bio_adjclose.skew()
kurtosis = df_bio_adjclose.kurtosis()
mini = df_bio_adjclose.min()
maxi = df_bio_adjclose.max()

col = ['mean', 'SD', 'skewness', 'Kurtosis', 'min', 'max', 'acf of lag 1']

acf_lag1 = []
for stock in bio_stocks:
    acf_lag1.append(pd.Series(df_bio_adjclose[stock]).autocorr())
acf_lag1 = pd.DataFrame(acf_lag1, index=bio_stocks)

df_stats_1 = pd.concat([mean, sd, skew, kurtosis, mini, maxi, acf_lag1], axis=1, ignore_index=True)
df_stats_1.columns = col
df_stats_1
# compute the statistics for volume
mean = df_bio_vol.mean()
sd = sqrt(df_bio_vol.var())
skew = df_bio_vol.skew()
kurtosis = df_bio_vol.kurtosis()
mini = df_bio_vol.min()
maxi = df_bio_vol.max()

acf_lag1 = []
for stock in bio_stocks:
    acf_lag1.append(pd.Series(df_bio_vol[stock]).autocorr())
acf_lag1 = pd.DataFrame(acf_lag1, index=bio_stocks)

df_stats_2 = pd.concat([mean, sd, skew, kurtosis, mini, maxi, acf_lag1], axis=1, ignore_index=True)
df_stats_2.columns = col
df_stats_2

,mean,SD,skewness,Kurtosis,min,max,acf of lag 1
BGNE,3.043567e+05,2.952018e+05,8.402415,108.316926,50900.0,5213400.0,0.307346
RPRX,2.139936e+06,1.561052e+06,7.743125,95.578870,307800.0,27619300.0,0.375624
ARGX,2.075210e+05,1.500386e+05,5.764484,65.695805,23600.0,2641900.0,0.396731
MRNA,8.587549e+06,1.157172e+07,4.287469,27.604352,272800.0,125130400.0,0.713378
HLN,8.330836e+06,2.645869e+06,1.178280,18.578602,1275800.0,36347500.0,0.760669
ZTS,2.005315e+06,8.403686e+05,1.676364,4.739723,417400.0,8065500.0,0.511973
VRTX,1.572511e+06,9.400755e+05,6.736403,89.921806,300500.0,17493000.0,0.405004
GILD,8.439954e+06,6.180807e+06,5.778081,54.311537,1965800.0,94348500.0,0.614391


In [11]:
from gurobipy import *
def optimize_portfolio(prices):
    # Calculate daily returns and statistics
    returns = prices.pct_change().dropna()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()

    # Number of assets
    num_assets = len(mean_returns)

    # Create a new model
    m = Model("portfolio_optimization")

    # Add variables
    x = m.addVars(num_assets, lb=0, ub=1, name="x")

    # Align indices for matrix multiplication
    portfolio_return = quicksum(mean_returns[i] * x[i] for i in range(num_assets))

    # Convert the covariance matrix to a 2D list for Gurobi
    cov_matrix_list = cov_matrix.values.tolist()
    portfolio_variance = x.prod(cov_matrix_list, x)

    # Set objective: maximize return and minimize variance
    m.setObjective(portfolio_return - portfolio_variance, GRB.MAXIMIZE)

    # Add constraint: sum of x_i (portfolio weights) must equal 1
    m.addConstr(x.sum() == 1, "budget")

    # Optimize the model
    m.optimize()

    # Decision making based on the optimization
    decisions = {}
    for i in range(num_assets):
        stock = prices.columns[i]
        if x[i].x > mean_returns[stock]:  # Example condition, replace with your logic
            decisions[stock] = "buy more"
        elif x[i].x < mean_returns[stock]:  # Example condition, replace with your logic
            decisions[stock] = "sell"
        else:
            decisions[stock] = "keep"

    return decisions

# Run the optimization
decisions = optimize_portfolio(df_bio_adjclose)
print(decisions)

/var/folders/z5/102n99n926x7011cmbz3_q7w0000gn/T/ipykernel_23665/2289943983.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  portfolio_return = quicksum(mean_returns[i] * x[i] for i in range(num_assets))


TypeError: tupledict.prod requires a dictionary

In [10]:
df_bio_adjclose

,BGNE,RPRX,ARGX,MRNA,HLN,ZTS,VRTX,GILD
2018-10-31,125.940002,42.761496,80.010002,16.743000,6.293415,87.021736,169.460007,55.752144
2018-11-01,130.089996,42.761496,96.050003,16.743000,6.293415,90.979454,175.190002,57.493881
2018-11-02,126.500000,42.761496,96.199997,16.743000,6.293415,89.550804,173.289993,56.782482
2018-11-05,126.970001,42.761496,97.910004,16.743000,6.293415,89.502541,174.589996,56.896965
2018-11-06,124.650002,42.761496,100.500000,16.743000,6.293415,89.541153,175.229996,57.526596
...,...,...,...,...,...,...,...,...
2023-10-24,172.960007,27.639999,481.200012,79.760002,8.260000,166.720825,369.380005,78.150002
2023-10-25,166.479996,27.299999,472.000000,76.760002,8.220000,163.279068,363.040009,78.389999
2023-10-26,165.850006,27.250000,470.429993,75.980003,7.990000,157.692444,361.279999,78.500000
2023-10-27,170.449997,26.209999,459.869995,71.910004,7.780000,155.657318,355.279999,76.620003
